In [26]:
# Instalar la librería Pulp, Pyomo, Gurobi
from pulp import *

In [27]:
#SETS
i_fabricas = ['i1', 'i2']
j_bodegas = ['j1', 'j2']
k_cds = ['k1', 'k2']
l_clientes = ['l1', 'l2', 'l3']

In [28]:
# Parámetros
oferta_i = {'i1': 100, 'i2': 100}
capacidad_b = {'j1': 80, 'j2': 100}
capacidad_cd = {'k1': 100, 'k2': 100}
dem = {'l1': 40, 'l2': 80, 'l3': 80}

costo_i_j = {('i1', 'j1'): 40, ('i1', 'j2'): 30,
             ('i2', 'j1'): 40, ('i2', 'j2'): 50}

costo_j_k = {('j1', 'k1'): 50, ('j1', 'k2'): 70,
             ('j2', 'k1'): 80, ('j2', 'k2'): 60}

costo_k_l = {('k1', 'l1'): 60, ('k1', 'l2'): 40, ('k1', 'l3'): 1e10,
             ('k2', 'l1'): 1e10, ('k2', 'l2'): 20, ('k2', 'l3'): 90}

costo_i_l = {('i1', 'l1'): 140, ('i1', 'l2'): 1e10, ('i1', 'l3'): 1e10,
             ('i2', 'l1'): 1e10, ('i2', 'l2'): 1e10, ('i2', 'l3'): 150}

In [29]:
# Variables de decisión
x = LpVariable.dicts("x", ((i, j) for i in i_fabricas for j in j_bodegas), lowBound=0)
y = LpVariable.dicts("y", ((j, k) for j in j_bodegas for k in k_cds),lowBound=0)
z = LpVariable.dicts("z", ((k, l) for k in k_cds for l in l_clientes), lowBound=0)
w = LpVariable.dicts("w", ((i, l) for i in i_fabricas for l in l_clientes), lowBound=0)
c23 = LpVariable("c23",lowBound=0)

In [30]:
# Problema de PL
productos_enviados = LpProblem("productos_enviados",LpMinimize)

In [31]:
# Funcion Objetivo
productos_enviados += (pulp.lpSum(x[i, j] * costo_i_j[i, j] for i in i_fabricas for j in j_bodegas) +
          pulp.lpSum(y[j, k] * costo_j_k[j, k] for j in j_bodegas for k in k_cds) +
          pulp.lpSum(z[k, l] * costo_k_l[k, l] for k in k_cds for l in l_clientes) +
          pulp.lpSum(w[i, l] * costo_i_l[i, l] for i in i_fabricas for l in l_clientes) +
          20 * c23)


In [32]:
# Restricciones
# R1: Oferta fabrica i
for i in i_fabricas:
    productos_enviados += lpSum(x[i, j] for j in j_bodegas) <= oferta_i[i]

# R2: Capacidad de las bodegas
for j in j_bodegas:
    productos_enviados += pulp.lpSum(x[i, j] for i in i_fabricas) <= capacidad_b[j]

# R3: Capacidad CD
for k in k_cds:
    productos_enviados += pulp.lpSum(y[j, k] for j in j_bodegas) <= capacidad_cd[k]
# R4: Demanda cliente 1
for l in l_clientes:
  productos_enviados += pulp.lpSum(z[k,l] for k in k_cds) + pulp.lpSum(w[i,l] for i in i_fabricas) >= 40

# R5: Demanda cliente 2
for l in l_clientes:
  productos_enviados+= pulp.lpSum(z[k,l] for k in k_cds) + pulp.lpSum(w[i,l] for i in i_fabricas) >= 80 + c23

# R6: Demanda cliente 3
for l in l_clientes:
  productos_enviados += pulp.lpSum(z[k,l] for k in k_cds) + pulp.lpSum(w[i,l] for i in i_fabricas) + c23 >= 80

# R7: Equilibrio bodegas
for j in j_bodegas:
    productos_enviados += pulp.lpSum(x[i, j] for i in i_fabricas) == pulp.lpSum(y[j, k] for k in k_cds)

# R8: Equilibrio CDs
for k in k_cds:
    productos_enviados += pulp.lpSum(y[j, k] for j in j_bodegas) == pulp.lpSum(z[k, l] for l in l_clientes)


In [33]:
# Resolver y mostrar los resultados
# Imprimir resultados
productos_enviados.solve()

print("Estado:", LpStatus[productos_enviados.status])
print("Beneficio máximo:", value(productos_enviados.objective))
for i in productos_enviados.variables():
    print(i.name, "=", i.varValue)

Estado: Optimal
Beneficio máximo: 32000.0
c23 = 0.0
w_('i1',_'l1') = 80.0
w_('i1',_'l2') = 0.0
w_('i1',_'l3') = 0.0
w_('i2',_'l1') = 0.0
w_('i2',_'l2') = 0.0
w_('i2',_'l3') = 80.0
x_('i1',_'j1') = 0.0
x_('i1',_'j2') = 80.0
x_('i2',_'j1') = 0.0
x_('i2',_'j2') = 0.0
y_('j1',_'k1') = 0.0
y_('j1',_'k2') = 0.0
y_('j2',_'k1') = 0.0
y_('j2',_'k2') = 80.0
z_('k1',_'l1') = 0.0
z_('k1',_'l2') = 0.0
z_('k1',_'l3') = 0.0
z_('k2',_'l1') = 0.0
z_('k2',_'l2') = 80.0
z_('k2',_'l3') = 0.0
